In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append('../')

# Custom functions
from src.data_split import create_masks_data_split_per_months
from src.paths import PREPROCESSED_DATA_DIR

In [4]:
import numpy as np
import pandas as pd
import hopsworks


In [5]:
df = pd.read_parquet(PREPROCESSED_DATA_DIR / 'modeling_dataset.parquet')

In [6]:
train_masks, val_masks, test_masks = create_masks_data_split_per_months(df,
                                                                        n_splits=3,
                                                                        offset_trainval=5,
                                                                        offset_test=1)

In [7]:
cat_vars = ['errors','use_chip','foreign_transaction', 'morning', 'afternoon','day_week']
high_cat_vars = ['mcc']
num_vars = ['amount','amount_log','recency','frequency','monetary']

In [8]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing._target_encoder import TargetEncoder
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

import xgboost as xgb


class TopCategoriesEncoder(BaseEstimator, TransformerMixin):
  def __init__(self, n=5):
    self.n = n
    self.top_categories = []

  def fit(self, X, y=None):
    # Calculate the proportion of the positive class for each category
    
    proportions = self.group_mean(X, y)

    # Sort the categories by the proportion of the positive class
    sorted_categories = dict(sorted(proportions.items(), key=lambda item: item[1], reverse=True))

    # Store the top n categories
    self.top_categories = list(sorted_categories.keys())[:self.n]

    return self
    
  def transform(self, X):
    # For each top category, add a new binary feature to X
    X = X.flatten()
    X_trans = np.zeros((X.shape[0], len(self.top_categories)))
    for i, category in enumerate(self.top_categories):
    
      X_trans[:, i] = (X == category)

    return X_trans

  @staticmethod
  def group_mean(X, y):
    # Get unique categories
    categories = np.unique(X)
    X = X.flatten()

    # Compute mean for each category
    means = {category: np.mean(y[X == category]) for category in categories}
  
    return means
    
class AddIsolationForestAnomalyScore(BaseEstimator, TransformerMixin):
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)
    
  def fit(self, X, y=None):
    n_estimators = getattr(self, 'n_estimators', 100)
    
    self.iforest = IsolationForest(n_estimators=n_estimators)
    self.iforest.fit(X)
    return self

  def transform(self, X):
    anomaly_scores = self.iforest.decision_function(X).reshape(-1, 1)
    return np.hstack([X, anomaly_scores])
      
      
class CatNarrow(BaseEstimator, TransformerMixin):
  "Narrows the categories of a categorical variable based on a threshold."

  def __init__(self, threshold: float=0.05):
    self.threshold = threshold
    
  def fit(self, X, y=None):
    X_ = np.array(X).astype(str)
    self.dc_filtered  = []
    
    # For each column, check the frequency of each category and filter based on threshold
    for c in range(X_.shape[1]):
      X_aux = X_[:, c]
      unique, counts = np.unique(X_aux, return_counts=True)
      counts_normalized = counts / sum(counts)
      dc = dict(zip(unique, counts_normalized))
      dc_filtered_ = {k: v for k, v in dc.items() if v >= self.threshold}  # Only keep categories with frequency >= threshold
      dc_filtered_ = np.array(list(dc_filtered_.keys())).reshape(-1,1)
      self.dc_filtered.append(dc_filtered_)
    
    return self
  
  def transform(self, X):
    """Replace categories with frequency < threshold by 'undefined'.
    
    Args:
      X (np.ndarray): The input array containing categories.
      
    Returns:
      X_trans (np.ndarray): The transformed array with replaced categories.
    """
       
    X_ = np.array(X).astype(str)
    list_aux = []
    
    for c in range(X_.shape[1]):  
      X_aux = np.array(X_[:, c]).reshape(-1,1)
      dc_filtered_ = self.dc_filtered[c]
      np.place(X_aux, np.isin(X_aux, dc_filtered_, invert = True), ['undefined']) # Replace categories with frequency < threshold by 'undefined'
      list_aux.append(X_aux)
    
    X_trans = np.concatenate(list_aux, axis = 1)
    return X_trans
  


num_transformer = Pipeline(steps = [
                                    ('imputer', SimpleImputer(strategy='median')),
                                    ('scaler', StandardScaler())
                                    ]
                          )

cat_transformer = Pipeline(steps = [
                                    ('imputer', SimpleImputer(
                                                              strategy='constant',
                                                              fill_value=-9999
                                                              )
                                    ),
                                    ('catnarrow', CatNarrow(threshold = 0.05)),
                                    ('target_enc', TargetEncoder()),
                                    ('scaler', StandardScaler())
                                            ]
                                   )

high_cat_transformer = Pipeline(steps = [
                                    ('imputer', SimpleImputer(
                                                              strategy='constant',
                                                              fill_value=-9999
                                                              )
                                    ),
                                    ('topcat_enc', TopCategoriesEncoder(n=3)),
                                    ('target_enc', TargetEncoder()),
                                    ('scaler', StandardScaler())
                                            ]
                                   )
                                   

processing = ColumnTransformer(transformers=[
                                            ('cat', cat_transformer, cat_vars),
                                            ('high_cat', high_cat_transformer, high_cat_vars),
                                            ('num', num_transformer, num_vars)
                                            ],
                               remainder='drop'
                              )
pipe = Pipeline(steps=[
                      ('processing', processing),
                      ('isolation_forest', AddIsolationForestAnomalyScore(n_estimators=100))
                       ]
              )

In [9]:
trainval_masks = train_masks + val_masks

In [10]:
split = 2


X_train = df.drop(columns=['fraud']).loc[trainval_masks[split]]
y_train = df['fraud'].loc[trainval_masks[split]]


X_test = df.drop(columns=['fraud']).loc[test_masks[split]]
y_test = df['fraud'].loc[test_masks[split]]


In [11]:
pipeline_training = pipe.fit(X_train, y_train)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [12]:
pipeline_training

Pipeline(steps=[('processing',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-9999,
                                                                                 strategy='constant')),
                                                                  ('catnarrow',
                                                                   CatNarrow()),
                                                                  ('target_enc',
                                                                   TargetEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['errors', 'use_chip',
                                                   'foreign_transaction',
                                                   'morning', 'afternoon',
                                                   'day_week']),
                                                 ('high_cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-9999,
                                                                                 strategy='constant')),
                                                                  ('topcat_enc',
                                                                   TopCategoriesEncoder(n=3)),
                                                                  ('target_enc',
                                                                   TargetEncoder()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['mcc']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['amount', 'amount_log',
                                                   'recency', 'frequency',
                                                   'monetary'])])),
                ('isolation_forest', AddIsolationForestAnomalyScore())])

In [13]:
X_train_trans = pipeline_training.transform(X_train)
X_test_trans = pipeline_training.transform(X_test)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [15]:
m0_rf = RandomForestClassifier(n_estimators=500, random_state=123, n_jobs=-1)
m0_rf.fit(X_train_trans, y_train)
pred_m0_rf = m0_rf.predict_proba(X_test_trans)[:, 1]


In [16]:

def ave_savings_score(y_true, score, amount, cf=5):
    #TODO: update description
    """Savings score.
   
    Parameters
    ----------
    y_true : array-like or label indicator matrix
        Ground truth (correct) labels.
    score : array-like or label indicator matrix
        Probabilities returned by a classifier.
    amount : array-like of shape = [n_samples, 4]
        Cost matrix of the classification problem
        Where the columns represents the costs of: false positives, false negatives,
        true positives and true negatives, for each example.
    cf    : float, default=10
        Fixed cost of investigation.
    Returns
    -------
    expected savings : float
        Savings of a using y_pred on y_true with cost-matrix cost-mat
        The best performance is 1.

    """

    ave_savings = np.sum(y_true*score*amount - score*cf)/np.sum(y_true*amount)
  
    return ave_savings


def savings_score(y_true, amount, cf=5):
    #TODO: update description
    """Savings score.
   
    Parameters
    ----------
    y_true : array-like or label indicator matrix
        Ground truth (correct) labels.
    amount : array-like of shape = [n_samples, 4]
        Cost matrix of the classification problem
        Where the columns represents the costs of: false positives, false negatives,
        true positives and true negatives, for each example.
    cf    : float, default=10
        Fixed cost of investigation.
    Returns
    -------
    expected savings : float
        Savings of a using y_pred on y_true with cost-matrix cost-mat
        The best performance is 1.

    """

    savings = np.sum(y_true*amount - len(y_true)*cf)/np.sum(y_true*amount)
  
    return savings


In [17]:
ix_sorted_top_rf = np.argsort(pred_m0_rf)[::-1]
topk = 0.01
y_test_ix = y_test.iloc[ix_sorted_top_rf][:int(topk*len(y_test))]
amount_test_ix = X_test['amount'].iloc[ix_sorted_top_rf][:int(topk*len(y_test))]

print('ROC-AUC: ', roc_auc_score(y_test, pred_m0_rf))
print('PR-AUC: ', average_precision_score(y_test, pred_m0_rf))
print('Expected Savings: ', ave_savings_score(y_test, pred_m0_rf, X_test['amount'], cf=10))
print('Uplift: ', average_precision_score(y_test, pred_m0_rf)/np.mean(y_test))
print('F1-score: ', f1_score(y_test, pred_m0_rf > 0.5))
print('Recall Top-1%: ',100*np.sum(y_test.iloc[ix_sorted_top_rf][:int(topk*len(y_test))])/np.sum(y_test))
print('Precision Top-1%', 100*np.mean(y_test.iloc[ix_sorted_top_rf][:int(topk*len(y_test))]))
print('Savings Top-1%: ', savings_score(y_test.iloc[ix_sorted_top_rf][:int(topk*len(y_test))], 
                                        X_test['amount'].iloc[ix_sorted_top_rf][:int(topk*len(y_test))]))
print('Recall Monetary Top-1%: ',100*np.sum(y_test_ix*amount_test_ix)/np.sum(y_test*X_test['amount']))

ROC-AUC:  0.9992751414953388
PR-AUC:  0.5999905348940753
Expected Savings:  0.30118410362652204
Uplift:  559.2639046467199
F1-score:  0.425531914893617
Recall Top-1%:  100.0
Precision Top-1% 10.749185667752444
Savings Top-1%:  -118.64298408636218
Recall Monetary Top-1%:  100.00000000000001


In [18]:

m0_xgb = xgb.XGBClassifier(n_estimators=100,
                           max_depth=5,
                           learning_rate=0.1,
                           random_state=123, 
                           n_jobs=-1
                           
                           )
m0_xgb.fit(X_train_trans, y_train)
pred_m0_xgb = m0_xgb.predict_proba(X_test_trans)[:, 1]


In [19]:
ix_sorted_top = np.argsort(pred_m0_xgb)[::-1]
topk = 0.01
y_test_ix = y_test.iloc[ix_sorted_top][:int(topk*len(y_test))]
amount_test_ix = X_test['amount'].iloc[ix_sorted_top][:int(topk*len(y_test))]

print('ROC-AUC: ', roc_auc_score(y_test, pred_m0_xgb))
print('PR-AUC: ', average_precision_score(y_test, pred_m0_xgb))
print('Expected Savings: ', ave_savings_score(y_test, pred_m0_xgb, X_test['amount']))
print('Uplift: ', average_precision_score(y_test, pred_m0_xgb)/np.mean(y_test))
print('F1-score: ', f1_score(y_test, pred_m0_xgb > 0.5))
print('Recall Top-1%: ', 100*np.sum(y_test.iloc[ix_sorted_top][:int(topk*len(y_test))])/np.sum(y_test))
print('Precision Top-1%', 100*np.mean(y_test.iloc[ix_sorted_top][:int(topk*len(y_test))]))
print('Recall Monetary Top-1%: ',100*np.sum(y_test_ix*amount_test_ix)/np.sum(y_test*X_test['amount']))


ROC-AUC:  0.999301768950612
PR-AUC:  0.5670612122378502
Expected Savings:  0.37237734525688027
Uplift:  528.5697844980689
F1-score:  0.43636363636363634
Recall Top-1%:  100.0
Precision Top-1% 10.749185667752444
Recall Monetary Top-1%:  100.0


In [20]:
import optuna
from sklearn.metrics import average_precision_score
def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "n_estimators": trial.suggest_int("n_estimators", 2, 50),
        "max_depth": trial.suggest_int("max_depth", 2, 5),
        "learning_rate": trial.suggest_float("learning_rate", 0.10, 1.0),
    }
       
    train_masks, val_masks, test_masks = create_masks_data_split_per_months(df,
                                                                            n_splits=3,
                                                                            offset_trainval=5,
                                                                            offset_test=1)
    scores = []
    
    train_masks = train_masks + val_masks
    for i in range(3):
        
        # split data for training and validation
        X_train = df.drop(columns=['fraud']).loc[train_masks[i]]
        y_train = df['fraud'].loc[train_masks[i]]
        
        X_val = df.drop(columns=['fraud']).loc[test_masks[i]]
        y_val = df['fraud'].loc[test_masks[i]]
        

        pipeline_training = pipe.fit(X_train, y_train)
        
        X_train = pipeline_training.transform(X_train)

        X_val = pipeline_training.transform(X_val)

        
        # train the model
        m0_xgb = xgb.XGBClassifier(**hyperparams,
                                    random_state=123, 
                                    n_jobs=-1
                                    )
        
        
        m0_xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        
        
        # evaluate the model
        y_pred = m0_xgb.predict_proba(X_val)[:, 1]
        ap = ave_savings_score(y_val, y_pred, df.drop(columns=['fraud']).loc[test_masks[i]]['amount'], cf=10)

        scores.append(ap)
   
    # Return the mean score
    return np.array(scores).mean()

In [21]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

[I 2024-02-29 20:39:20,962] A new study created in memory with name: no-name-98a2362f-ad72-4c4c-9e83-b49804ad72ab
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for m

In [22]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'n_estimators': 27, 'max_depth': 5, 'learning_rate': 0.8962484708293166}


In [23]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_n_estimators,state
0,0,0.285586,2024-02-29 20:39:20.962879,2024-02-29 20:39:27.157831,0 days 00:00:06.194952,0.849096,5,40,COMPLETE
1,1,0.309632,2024-02-29 20:39:27.157831,2024-02-29 20:39:33.311602,0 days 00:00:06.153771,0.972736,5,18,COMPLETE
2,2,-1.274205,2024-02-29 20:39:33.312602,2024-02-29 20:39:39.328717,0 days 00:00:06.016115,0.174824,2,12,COMPLETE
3,3,0.171993,2024-02-29 20:39:39.329717,2024-02-29 20:39:45.360466,0 days 00:00:06.030749,0.489049,2,11,COMPLETE
4,4,0.307193,2024-02-29 20:39:45.361464,2024-02-29 20:39:51.418539,0 days 00:00:06.057075,0.418457,4,21,COMPLETE
5,5,0.342572,2024-02-29 20:39:51.419539,2024-02-29 20:39:57.725131,0 days 00:00:06.305592,0.549250,5,43,COMPLETE
6,6,0.227947,2024-02-29 20:39:57.726131,2024-02-29 20:40:03.798296,0 days 00:00:06.072165,0.225523,2,36,COMPLETE
7,7,0.295077,2024-02-29 20:40:03.799293,2024-02-29 20:40:09.920651,0 days 00:00:06.121358,0.776345,5,41,COMPLETE
8,8,0.330412,2024-02-29 20:40:09.921662,2024-02-29 20:40:15.920697,0 days 00:00:05.999035,0.642479,4,24,COMPLETE
9,9,-1.427305,2024-02-29 20:40:15.920697,2024-02-29 20:40:21.899375,0 days 00:00:05.978678,0.147435,5,13,COMPLETE


In [24]:
m0_xgb = xgb.XGBClassifier(**best_params,
                            random_state=123, 
                            n_jobs=-1
                            )

m0_xgb.fit(X_train_trans, y_train, verbose=False)
pred_m0_xgb = m0_xgb.predict_proba(X_test_trans)[:, 1]

In [25]:
ix_sorted_top = np.argsort(pred_m0_xgb)[::-1]
topk = 0.01
y_test_ix = y_test.iloc[ix_sorted_top][:int(topk*len(y_test))]
amount_test_ix = X_test['amount'].iloc[ix_sorted_top][:int(topk*len(y_test))]

print('ROC-AUC: ', roc_auc_score(y_test, pred_m0_xgb))
print('PR-AUC: ', average_precision_score(y_test, pred_m0_xgb))
print('Expected Savings: ', ave_savings_score(y_test, pred_m0_xgb, X_test['amount']))
print('Uplift: ', average_precision_score(y_test, pred_m0_xgb)/np.mean(y_test))
print('F1-score: ', f1_score(y_test, pred_m0_xgb > 0.5))
print('Recall Top-1%: ', 100*np.sum(y_test.iloc[ix_sorted_top][:int(topk*len(y_test))])/np.sum(y_test))
print('Precision Top-1%', 100*np.mean(y_test.iloc[ix_sorted_top][:int(topk*len(y_test))]))
print('Recall Monetary Top-1%: ',100*np.sum(y_test_ix*amount_test_ix)/np.sum(y_test*X_test['amount']))

ROC-AUC:  0.9984644834125747
PR-AUC:  0.5166963623722588
Expected Savings:  0.3368137257199589
Uplift:  481.623639593051
F1-score:  0.3728813559322034
Recall Top-1%:  96.96969696969697
Precision Top-1% 10.423452768729643
Recall Monetary Top-1%:  99.66918522580711


In [28]:
np.sum(y_test)/60

0.55